### Задание 1

Для нашего пайплайна (Case1) поэкспериментировать с разными моделями:
    - бустинг
    - логистическая регрессия

In [67]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix

import itertools

import matplotlib.pyplot as plt

%matplotlib inline

Ссылка на google drive: https://drive.google.com/file/d/1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91

In [68]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Есть как категориальные, так и вещественные признаки. Поле CustomerId нужно будет удалить. 

Посмотрим на распределение классов:

In [69]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Давайте построим модель. Сразу же будем работать с использованием sklearn pipeline

In [70]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

- Категориальные признаки закодируем с помощью OneHotEncoding
- Вещественные оставим пока как есть

In [71]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [72]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Зададим списки признаков

In [73]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

Теперь нам нужно под каждый признак создать трансформер и объединить их в список.

In [74]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [75]:
feats = FeatureUnion(final_transformers)

In [76]:
metrics = pd.DataFrame(columns=['Model', 'Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])

In [77]:
def result(model):

    pipeline = Pipeline([
        ('features', feats),
        ('classifier', model),
    ])
    
    # обучим наш пайплайн
    pipeline.fit(X_train, y_train)
    
    # наши прогнозы для тестовой выборки
    preds = pipeline.predict_proba(X_test)[:, 1]
    
    #Также нам нужно от вероятностей перейти к меткам классов. 
    #Для этого нужно подобрать порог, после которого мы считаем, что объект можно отнести к классу 1 
    #(если вероятность больше порога - размечаем объект как класс 1, если нет - класс 0)
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)

    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    
    roc_auc = roc_auc_score(y_test, preds)
    
    cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])
    TN = cnf_matrix[0][0]
    FP = cnf_matrix[0][1]
    FN = cnf_matrix[1][0]
    TP = cnf_matrix[1][1]

    return model, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc, cnf_matrix

##### Градиентный бустинг

In [78]:
model_info = result(GradientBoostingClassifier(random_state=42))

In [79]:
metrics = metrics.append({
    'Model': model_info[0].__class__.__name__,
    'Threshold': model_info[1],
    'F-Score': model_info[2],
    'Precision': model_info[3],
    'Recall': model_info[4],
    'ROC AUC': model_info[5]}, ignore_index=True)
metrics

,Model,Threshold,F-Score,Precision,Recall,ROC AUC
0,GradientBoostingClassifier,0.408508,0.646121,0.703704,0.59725,0.875746


##### Логистическая регрессия

In [80]:
model_info = result(LogisticRegression(random_state=42))

In [81]:
metrics = metrics.append({
    'Model': model_info[0].__class__.__name__,
    'Threshold': model_info[1],
    'F-Score': model_info[2],
    'Precision': model_info[3],
    'Recall': model_info[4],
    'ROC AUC': model_info[5]}, ignore_index=True)
metrics

,Model,Threshold,F-Score,Precision,Recall,ROC AUC
0,GradientBoostingClassifier,0.408508,0.646121,0.703704,0.59725,0.875746
1,LogisticRegression,0.289522,0.509700,0.462400,0.56778,0.772077


### Задание 2

Отобрать лучшую модель по метрикам (какая по вашему мнению здесь наиболее подходящая ML-метрика)

In [82]:
metrics.loc[metrics['F-Score'] == metrics['F-Score'].max()]

,Model,Threshold,F-Score,Precision,Recall,ROC AUC
0,GradientBoostingClassifier,0.408508,0.646121,0.703704,0.59725,0.875746


### Задание 3

Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2:
    - 1 доллар на удержание
    - 2 доллара - с каждого правильно классифицированного (True Positive)

In [83]:
_, _, _, _, _, _, cnf_matrix = result(GradientBoostingClassifier(random_state=42))

In [84]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]

retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

175

### Задание 4

*Провести подбор гиперпараметров лучшей модели по итогам 2-3

### Задание 5

*Еще раз провести оценку экономической эффективности